<a href="https://colab.research.google.com/github/maimuna-noshin/Solar_Prediction/blob/main/solarprediction_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import get_custom_objects
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pickle
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from tensorflow.keras.layers import Dense, Dropout, LSTM, Conv1D, BatchNormalization, Flatten
from sklearn.preprocessing import StandardScaler
from scipy.fftpack import fft
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [136]:
df=pd.read_csv("/content/updated_dataset.csv")

In [137]:
# Convert to datetime
df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day', 'Hour']])




In [138]:
# Selecting features (X) and target variables (Y)
features = ['Hour', 'Temperature_A','Pressure_A', 'Day','Month', 'Surface Albedo_A', 'Cloud Type_A']
targets = ['DNI_A']  # Target variables for all three locations

In [139]:

X = df[features].values
y = df[targets].values

In [140]:
# Keep timestamps before scaling
timestamps = df[['Month', 'Day', 'Hour']]

In [141]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler(feature_range=(0, 1000))
X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)
# Save the feature scaler
with open("scaler_X.pkl", "wb") as f:
    pickle.dump(scaler_X, f)

# Save the target scaler
with open("scaler_y.pkl", "wb") as f:
    pickle.dump(scaler_y, f)

In [142]:
hours_tensor = tf.convert_to_tensor(df['Hour'].values, dtype=tf.float32)


In [143]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test, timestamps_train, timestamps_test = train_test_split(X_scaled, y_scaled,timestamps, test_size=0.2, random_state=42)

In [144]:
def custom_loss_fixed(y_true, y_pred):
    batch_size = tf.shape(y_true)[0]
    batch_hours = tf.gather(hours_tensor, tf.range(batch_size))  # Use hours_tensor instead of hours

    nighttime_mask = tf.logical_or(tf.less_equal(batch_hours, 8), tf.greater_equal(batch_hours, 18))
    nighttime_mask = tf.cast(nighttime_mask, tf.float32)

    mse_loss = tf.keras.losses.MSE(y_true, y_pred)
    penalty_factor = 50.0
    weighted_loss = mse_loss * (1 + (penalty_factor - 1) * nighttime_mask)

    return tf.reduce_mean(weighted_loss)




In [145]:
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(targets), activation='linear')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [146]:
model.compile(optimizer='adam', loss=custom_loss_fixed, metrics=['mae'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [147]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=8, validation_data=(X_test, y_test))

Epoch 1/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3353709.0000 - mae: 194.1918 - val_loss: 1517535.5000 - val_mae: 115.1331
Epoch 2/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 1589843.2500 - mae: 118.3901 - val_loss: 1370910.8750 - val_mae: 107.5927
Epoch 3/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1456403.0000 - mae: 107.8436 - val_loss: 1239561.5000 - val_mae: 93.0351
Epoch 4/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1333733.3750 - mae: 98.0960 - val_loss: 1210874.5000 - val_mae: 91.3765
Epoch 5/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1303910.2500 - mae: 97.0862 - val_loss: 1176177.3750 - val_mae: 90.3489
Epoch 6/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1249273.0000 - mae: 94.4662 - val_loss: 1182044.1250 - val_mae: 92.1190
Epoch 7/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1172865.3750 - mae: 90.2968 - val_loss: 1151360.5000 - val_mae: 87.3514
Epoch 8/10
876/876 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1241432.125

In [148]:
# Predict future solar irradiance for all three locations
y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)


55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [149]:
# Convert predictions into a DataFrame
predictions_df = timestamps_test.copy()

# Assign each column separately
predictions_df[['DNI_A']] = y_pred

In [150]:
# Print first 100 predictions with date and hour
print(predictions_df.head(100))
predictions_df.to_csv("solar_irradiance_predictions.csv", index=False)  # Saves to a CSV]

      Month  Day  Hour       DNI_A
6056      9   10     8  148.175308
5556      8   20    12  282.674957
5990      9    7    14  321.092865
7674     11   16    18  177.015640
3319      5   19     7  109.099396
...     ...  ...   ...         ...
6006      9    8     6    8.087282
8063     12    2    23    2.897856
14        1    1    14  478.830811
5306      8   10     2    1.438244
8524     12   22     4    1.004926

[100 rows x 4 columns]


In [151]:
model.save("solar_irradiance_predictions.keras")


model = keras.models.load_model(
    "solar_irradiance_predictions.keras",
    custom_objects={"custom_loss_fixed": custom_loss_fixed},
    safe_mode=False  # Allows deserialization of custom objects
)





In [152]:
#  Load the scalers used during training

with open("scaler_X.pkl", "rb") as f:
    scaler_X = pickle.load(f)
with open("scaler_y.pkl", "rb") as f:
    scaler_y = pickle.load(f)

In [153]:
# Generate timestamps for the first week of March 2025 (hourly)
date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")
future_df = pd.DataFrame({"datetime": date_range})

<ipython-input-153-0e223ba96d09>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start="2025-03-01", end="2025-03-07 23:00:00", freq="H")


In [154]:
#  Feature Engineering
future_df["day"] = future_df["datetime"].dt.day
future_df["month"] = future_df["datetime"].dt.month
future_df["hour"] = future_df["datetime"].dt.hour
future_df.drop(columns=["datetime"], inplace=True)

In [155]:
# Generate random values
future_df["Temperature_A"] = np.random.uniform(27, 30, size=168)  # Temperature between 15°C to 30°C
future_df["Pressure_A"] = np.random.uniform(1000, 1050, size=168)  # Pressure between 950-1050 hPa
future_df["Surface Albedo_A"] = np.random.uniform(0.1, 0.5, size=168)  # Albedo between 0.1 - 0.5
future_df["Cloud Type_A"] = np.random.randint(0, 4, size=168)  # Cloud types (0-3)

# Merge the first week’s random values with `future_df`
#future_df = future_df.merge(future_df, on=["Month", "Day"], how="left")

# Time-based encoding (sin/cos transformation)
#future_df["hour_sin"] = np.sin(2 * np.pi * future_df["hour"] / 24)
#future_df["hour_cos"] = np.cos(2 * np.pi * future_df["hour"] / 24)

# Select the same features used in training
#X_future = future_df[['hour_sin', 'hour_cos', 'Temperature_A', 'Pressure_A', 'day', 'Month', 'Surface Albedo_A', 'Cloud Type_A']].values

In [156]:
#  Select the same features used in training
X_future = future_df[['hour', 'Temperature_A','Pressure_A', 'day','month', 'Surface Albedo_A', 'Cloud Type_A']].values
#  Scale future data using the trained scaler
X_future_scaled = scaler_X.transform(X_future)
print(X_future_scaled)


[[0.         0.63584829 1.25611464 ... 0.18181818 0.96357312 0.22222222]
 [0.04347826 0.60151976 0.35048912 ... 0.18181818 2.42026451 0.        ]
 [0.08695652 0.59044547 1.66355813 ... 0.18181818 1.86901089 0.22222222]
 ...
 [0.91304348 0.65545849 0.55240681 ... 0.18181818 3.65113786 0.22222222]
 [0.95652174 0.60447763 0.54152899 ... 0.18181818 0.11682362 0.22222222]
 [1.         0.62956356 1.10815991 ... 0.18181818 3.22047686 0.        ]]


In [157]:
#  Make predictions
y_future_scaled = model.predict(X_future_scaled)

#  Inverse scale predictions
y_future = scaler_y.inverse_transform(y_future_scaled)
#  Set negative irradiance values to 0
y_future[y_future < 15] = 0

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [158]:
# Save predictions to DataFrame
future_df[['DNI_A']] = y_future

#  Save to CSV
future_df.to_csv("Future_Solar_Predictions_March2025.csv", index=False)

#Display first 10 predictions
print(future_df.head(10))

   day  month  hour  Temperature_A   Pressure_A  Surface Albedo_A  \
0    1      3     0      28.928654  1028.683439          0.188179   
1    1      3     1      27.809544  1001.514673          0.261013   
2    1      3     2      27.448522  1040.906744          0.233451   
3    1      3     3      29.319266  1014.481050          0.345836   
4    1      3     4      27.869334  1019.134334          0.230381   
5    1      3     5      27.694777  1010.269331          0.481468   
6    1      3     6      29.921404  1010.691317          0.328631   
7    1      3     7      29.255739  1009.526937          0.294309   
8    1      3     8      28.315013  1007.584922          0.274253   
9    1      3     9      28.581875  1008.974374          0.413074   

   Cloud Type_A       DNI_A  
0             2    0.000000  
1             0    0.000000  
2             2    0.000000  
3             0    0.000000  
4             3    0.000000  
5             0    0.000000  
6             0    0.000000  
